In [1]:
TRAIT = "Epilepsy"
CONDITION = 'Obesity'
QUESTION = f'What are the genetic factors related to {TRAIT} when considering the influence of {CONDITION}?'
print(f"Studied question: {QUESTION}")

# Gene symbol normalization may take a few minutes. Set to True for the final run after debugging.
NORMALIZE_GENE = True

Studied question: What are the genetic factors related to Epilepsy when considering the influence of Obesity?


# 1. Basic setups

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

In [3]:
import os
from utils import *

# data_root = './data'
data_root = '/media/techt/DATA'
dataset = 'GEO'
trait_subdir = "Epilepsy"
condition_subdir = "Obesity"
trait_accession_num = "GSE205661"
condition_accession_num = "GSE181339"
output_dir = './output'

trait_cohort_dir = os.path.join(data_root, dataset, trait_subdir, trait_accession_num)
condition_cohort_dir = os.path.join(data_root, dataset, condition_subdir, condition_accession_num)

In [4]:
trait_soft_file, trait_matrix_file = get_relevant_filepaths(trait_cohort_dir)
condition_soft_file, condition_matrix_file = get_relevant_filepaths(condition_cohort_dir)

In [5]:
trait_soft_file, trait_matrix_file

('/media/techt/DATA/GEO/Epilepsy/GSE205661/GSE205661_family.soft.gz',
 '/media/techt/DATA/GEO/Epilepsy/GSE205661/GSE205661-GPL13534_series_matrix.txt.gz')

# 2. Data proprocessing

## 2.1 Preparing the trait dataset

We are studying the association between some human traits and gene expression. Please read the below background information and decide whether this dataset is relevant, by answering the below questions:

(1) Does it contain gene expression data? Pure miRNA data are not usable.
(2) Does it contain human data about the trait '{TRAIT}'? If so, can we quantify this trait as a binary or continuous variable?
(3) Does it contain human data about the trait '{CONDITION}'? If so, can we quantify this trait as a binary or continuous variable?

In [6]:
background_prefixes = ['!Sample_geo_accession', '!Series_title', '!Series_summary', '!Series_overall_design', '!Sample_characteristics_ch1']

trait_background_info = get_background_info(trait_matrix_file, background_prefixes)
print(trait_background_info)

!Series_title	"Integrated analysis of expression profile and potential pathogenic mechanism of temporal lobe epilepsy with hippocampal sclerosis"
!Series_summary	"To investigate the potential pathogenic mechanism of temporal lobe epilepsy with hippocampal sclerosis (TLE+HS), we have employed analyzing of the expression profiles of microRNA/ mRNA/ lncRNA/ DNA methylation in brain tissues of hippocampal sclerosis (TLE+HS) patients. Brain tissues of six patients with TLE+HS and nine of normal temporal or parietal cortices (NTP) of patients undergoing internal decompression for traumatic brain injury (TBI) were collected. The total RNA was dephosphorylated, labeled, and hybridized to the Agilent Human miRNA Microarray, Release 19.0, 8x60K. The cDNA was labeled and hybridized to the Agilent LncRNA+mRNA Human Gene Expression Microarray V3.0，4x180K. For methylation detection, the DNA was labeled and hybridized to the Illumina 450K Infinium Methylation BeadChip. The raw data was extracted from

Focus on the clinical traits of the samples.

In [7]:
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
trait_clinical_data = get_clinical_data(trait_background_info, clinical_prefixes)

In [8]:
trait_clinical_data

,!Sample_geo_accession,GSM6216198,GSM6216199,GSM6216200,GSM6216201,GSM6216202,GSM6216203,GSM6216204,GSM6216205,GSM6216206,GSM6216207,GSM6216208,GSM6216209,GSM6216210,GSM6216211,GSM6216212
0,!Sample_characteristics_ch1,tissue: Hippocampus,tissue: Hippocampus,tissue: Hippocampus,tissue: Hippocampus,tissue: Hippocampus,tissue: Hippocampus,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Temporal lobe,tissue: Parietal lobe,tissue: Temporal lobe,tissue: Parietal lobe
1,!Sample_characteristics_ch1,gender: Female,gender: Male,gender: Male,gender: Female,gender: Female,gender: Male,gender: Female,gender: Male,gender: Female,gender: Male,gender: Male,gender: Male,gender: Male,gender: Male,gender: Male
2,!Sample_characteristics_ch1,age: 23y,age: 29y,age: 37y,age: 26y,age: 16y,age: 13y,age: 62y,age: 58y,age: 63y,age: 68y,age: 77y,age: 59y,age: 50y,age: 39y,age: 23y


Find the index of the row that records data about '{TRAIT}'. The index should start with 0. Choose a proper data type for the trait between 'binary' and 'continuous', and write a function that converts a value in the cell into that data type.

In [9]:
trait_row_id = 0

def decode_trait_var(value):
    if value == 'tissue: Hippocampus':
        return 1
    else:
        return 0

In [10]:
trait_feature_data = get_feature_data(trait_clinical_data, trait_row_id, TRAIT, decode_trait_var)

In [11]:
trait_gene_annotation = get_gene_annotation(trait_soft_file)

Read the gene annotation dataframe, to get the name of the column that records gene symbols.

In [12]:
trait_gene_annotation.head()

,ID,Name,AddressA_ID,AlleleA_ProbeSeq,AddressB_ID,AlleleB_ProbeSeq,Infinium_Design_Type,Next_Base,Color_Channel,Forward_Sequence,...,DMR,Enhancer,HMM_Island,Regulatory_Feature_Name,Regulatory_Feature_Group,DHS,RANGE_START,RANGE_END,RANGE_GB,SPOT_ID
0,cg00035864,cg00035864,31729416,AAAACACTAACAATCTTATCCACATAAACCCTTAAATTTATCTCAA...,NaN,NaN,II,NaN,NaN,AATCCAAAGATGATGGAGGAGTGCCCGCTCATGATGTGAAGTACCT...,...,NaN,NaN,NaN,NaN,NaN,NaN,8553009.0,8553132.0,NC_000024.9,NaN
1,cg00050873,cg00050873,32735311,ACAAAAAAACAACACACAACTATAATAATTTTTAAAATAAATAAAC...,31717405,ACGAAAAAACAACGCACAACTATAATAATTTTTAAAATAAATAAAC...,I,A,Red,TATCTCTGTCTGGCGAGGAGGCAACGCACAACTGTGGTGGTTTTTG...,...,NaN,NaN,Y:9973136-9976273,NaN,NaN,NaN,9363356.0,9363479.0,NC_000024.9,NaN
2,cg00061679,cg00061679,28780415,AAAACATTAAAAAACTAATTCACTACTATTTAATTACTTTATTTTC...,NaN,NaN,II,NaN,NaN,TCAACAAATGAGAGACATTGAAGAACTAATTCACTACTATTTGGTT...,...,NaN,NaN,NaN,NaN,NaN,NaN,25314171.0,25314294.0,NC_000024.9,NaN
3,cg00063477,cg00063477,16712347,TATTCTTCCACACAAAATACTAAACRTATATTTACAAAAATACTTC...,NaN,NaN,II,NaN,NaN,CTCCTGTACTTGTTCATTAAATAATGATTCCTTGGATATACCAAGT...,...,NaN,NaN,NaN,NaN,NaN,NaN,22741795.0,22741918.0,NC_000024.9,NaN
4,cg00121626,cg00121626,19779393,AAAACTAATAAAAATAACTTACAAACCAAATACTATACCCTACAAC...,NaN,NaN,II,NaN,NaN,AGGTGAATGAAGAGACTAATGGGAGTGGCTTGCAAGCCAGGTACTG...,...,NaN,NaN,NaN,NaN,NaN,NaN,21664296.0,21664419.0,NC_000024.9,NaN


In [13]:
trait_gene_name_col = 'UCSC_RefGene_Name'

In [14]:
trait_gene_mapping = get_gene_mapping(trait_gene_annotation, trait_gene_name_col)

Load tabular gene expression data

In [15]:
trait_genetic_data = get_genetic_data(trait_matrix_file)

In [16]:
trait_genetic_data = apply_gene_mapping(trait_genetic_data, trait_gene_mapping)

In [17]:
if NORMALIZE_GENE:
    trait_genetic_data = normalize_gene_symbols_in_index(trait_genetic_data)

15 input query terms found dup hits:	[('ABCC13', 2), ('ABCC6P1', 2), ('ABCC6P2', 3), ('ADAM6', 3), ('AGAP11', 2), ('ALOX12P2', 2), ('ANKR
71 input query terms found no hit:	['A2BP1', 'A2LD1', 'AACSL', 'AARS', 'ABP1', 'ACCN1', 'ACCN2', 'ACCN3', 'ACCN4', 'ACCN5', 'ACN9', 'AC
6 input query terms found dup hits:	[('ATP6AP1L', 2), ('ATXN8OS', 2), ('BAGE2', 2), ('BMS1P1', 2), ('BMS1P4', 2), ('BRD7P3', 2)]
364 input query terms found no hit:	['ARNTL2', 'ARPM1', 'ARPP-21', 'ARSE', 'ASAM', 'ASAP1IT1', 'ASFMR1', 'ASNA1', 'ATHL1', 'ATP5A1', 'AT
4 input query terms found dup hits:	[('C2orf27A', 2), ('C2orf83', 2), ('C3P1', 2), ('C5orf60', 2)]
642 input query terms found no hit:	['C17orf56', 'C17orf57', 'C17orf59', 'C17orf60', 'C17orf61', 'C17orf62', 'C17orf63', 'C17orf64', 'C1
9 input query terms found dup hits:	[('CATSPER2P1', 2), ('CCDC144NL', 2), ('CCT6P1', 2), ('CDR1', 2), ('CECR7', 2), ('CELP', 2), ('CHKB-
98 input query terms found no hit:	['CBARA1', 'CBWD1', 'CBWD2', 'CBWD3', 'CBWD5', 'CBWD

In [18]:
trait_merged_data = add_binary_feature(trait_genetic_data, trait_feature_data, TRAIT)
trait_merged_data = trait_merged_data.T
trait_merged_data

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAA1,AAAS,AACS,AADAC,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Epilepsy
GSM6216198,0.498235,0.909286,0.803357,0.881167,0.454591,0.887167,0.9117,0.157538,0.706483,0.92025,...,0.039643,0.460750,0.5251,0.332605,0.554000,0.519077,0.365947,0.522561,0.162056,1.0
GSM6216199,0.479765,0.900571,0.831571,0.874278,0.437136,0.895333,0.9172,0.157769,0.702603,0.91275,...,0.041286,0.236667,0.2783,0.323442,0.559333,0.517308,0.375368,0.512585,0.139722,1.0
GSM6216200,0.490882,0.900000,0.814643,0.844056,0.437409,0.872500,0.9123,0.153846,0.703810,0.91425,...,0.041214,0.229333,0.3173,0.331791,0.558600,0.508000,0.368316,0.517439,0.149056,1.0
GSM6216201,0.527765,0.900857,0.771929,0.903000,0.440500,0.804333,0.8902,0.164000,0.725345,0.92125,...,0.041000,0.448167,0.6069,0.332116,0.560267,0.528846,0.317737,0.517293,0.157000,1.0
GSM6216202,0.485706,0.895857,0.776429,0.838000,0.438864,0.842500,0.8885,0.161385,0.705621,0.88225,...,0.047000,0.398750,0.4872,0.325698,0.546733,0.519769,0.356000,0.514829,0.158444,1.0
GSM6216203,0.464529,0.896857,0.788929,0.863778,0.444364,0.863500,0.9017,0.153462,0.685948,0.92375,...,0.038929,0.239417,0.2949,0.322256,0.548467,0.514000,0.364158,0.513976,0.133444,1.0
GSM6216204,0.543765,0.905286,0.796500,0.889667,0.452727,0.876833,0.9155,0.175077,0.650776,0.94700,...,0.037857,0.435333,0.4943,0.327953,0.554667,0.507000,0.332474,0.512585,0.152444,0.0
GSM6216205,0.530529,0.895000,0.821571,0.890611,0.449409,0.875167,0.9178,0.157308,0.665466,0.92750,...,0.038071,0.243833,0.2748,0.323163,0.553467,0.500308,0.347474,0.516390,0.150389,0.0
GSM6216206,0.541882,0.920143,0.805357,0.851278,0.459182,0.860833,0.9167,0.158615,0.658121,0.91650,...,0.039143,0.461500,0.5138,0.336093,0.566867,0.526615,0.327684,0.525683,0.160889,0.0
GSM6216207,0.558471,0.877429,0.852857,0.864000,0.434091,0.912667,0.9123,0.156769,0.709310,0.92175,...,0.035643,0.257000,0.2312,0.346721,0.576800,0.506615,0.386579,0.515854,0.128111,0.0


In [19]:
biased = judge_binary_variable_biased(trait_merged_data, TRAIT, 0.1, 10)
if biased:
    print(f"The distribution of the trait \'{TRAIT}\' in this dataset is severely biased.")
else:
    print(f"The distribution of the trait \'{TRAIT}\' in this dataset is fine.")

The least common label is '1.0' with 6 occurrences. This represents 40.00% of the dataset.
The distribution of the trait 'Epilepsy' in this dataset is fine.


## 2.2 Preparing the condition dataset

We are studying the association between some human traits and gene expression. Please read the below background information and decide whether this dataset is relevant, by answering the below questions:

(1) Does it contain gene expression data? Pure miRNA data are not usable.
(2) Does it contain human data about the trait '{CONDITION}'? If so, can we quantify this trait as a binary or continuous variable?
(3) Does it contain human data about the trait '{TRAIT}'? If so, can we quantify this trait as a binary or continuous variable?

In [20]:
condition_background_info = get_background_info(condition_matrix_file, background_prefixes)
print(condition_background_info)

!Series_title	"Study of the usefulness of human peripheral blood mononuclear cells for the analysis of metabolic recovery after weight loss (METAHEALTH-TEST)"
!Series_summary	"The aim of this study is to design and validate a test, METAHEALTH-TEST, based on gene expression analysis in blood cells, to quickly and easily analyse metabolic health. This test will be used to analyse metabolic improvement in overweight/obese individuals and in metabolically obese normal-weight (MONW) individuals after undergoing a weight loss intervention and/or an intervention for improvement in eating habits and lifestyle. Obesity and its medical complications are a serious health problem today. Using peripheral blood mononuclear cells (PBMC) as an easily obtainable source of transcriptomic biomarkers would allow to deepen into the knowledge of adaptations in response to increased adiposity that occur in internal homeostatic tissues, without the need of using invasive biopsies. Moreover, if PBMC were able 

Focus on the clinical traits of the samples.

In [21]:
condition_clinical_data = get_clinical_data(condition_background_info, clinical_prefixes)
condition_clinical_data.head()

,!Sample_geo_accession,GSM5494930,GSM5494931,GSM5494932,GSM5494933,GSM5494934,GSM5494935,GSM5494936,GSM5494937,GSM5494938,...,GSM5494998,GSM5494999,GSM5495000,GSM5495001,GSM5495002,GSM5495003,GSM5495004,GSM5495005,GSM5495006,GSM5495007
0,!Sample_characteristics_ch1,gender: Man,gender: Man,gender: Woman,gender: Woman,gender: Woman,gender: Man,gender: Woman,gender: Woman,gender: Woman,...,gender: Woman,gender: Woman,gender: Man,gender: Man,gender: Man,gender: Man,gender: Man,gender: Woman,gender: Man,gender: Man
1,!Sample_characteristics_ch1,group: NW,group: OW/OB,group: MONW,group: OW/OB,group: NW,group: OW/OB,group: OW/OB,group: OW/OB,group: NW,...,group: OW/OB,group: OW/OB,group: OW/OB,group: NW,group: OW/OB,group: NW,group: OW/OB,group: OW/OB,group: OW/OB,group: NW
2,!Sample_characteristics_ch1,age: 21,age: 23,age: 10,age: 17,age: 11,age: 1,age: 18,age: 10,age: 12,...,age: 2,age: 2,age: 30,age: 17,age: 30,age: 19,age: 30,age: 2,age: 4,age: 19
3,!Sample_characteristics_ch1,fasting time: 6hr,fasting time: 4hr,fasting time: 6hr,fasting time: 4hr,fasting time: 4hr,fasting time: 6hr,fasting time: 4hr,fasting time: 6hr,fasting time: 4hr,...,fasting time: 6hr,fasting time: 4hr,fasting time: 4hr,fasting time: 6hr,fasting time: 6hr,fasting time: 4hr,fasting time: 4hr,fasting time: 6hr,fasting time: 4hr,fasting time: 6hr
4,!Sample_characteristics_ch1,timepoint: 0months,timepoint: 6months,timepoint: 0months,timepoint: 0months,timepoint: 0months,timepoint: 6months,timepoint: 6months,timepoint: 0months,timepoint: 0months,...,timepoint: 6months,timepoint: 0months,timepoint: 0months,timepoint: 0months,timepoint: 6months,timepoint: 0months,timepoint: 6months,timepoint: 0months,timepoint: 0months,timepoint: 0months


Find the index of the row that records data about '{CONDITION}'. The index should start with 0. Choose a proper data type for the trait between 'binary' and 'continuous', and write a function that converts a value in the cell into that data type.

In [22]:
condition_row_id = 1

def decode_condition_var(value):
    if value == 'group: OW/OB':
        return 1
    else:
        return 0

In [23]:
condition_feature_data = get_feature_data(condition_clinical_data, condition_row_id, CONDITION, decode_condition_var)

In [24]:
condition_gene_annotation = get_gene_annotation(condition_soft_file)

Read the gene annotation dataframe, to get the name of the column that records gene symbols.

In [25]:
condition_gene_annotation.head()

,ID,COL,ROW,NAME,SPOT_ID,CONTROL_TYPE,REFSEQ,GB_ACC,LOCUSLINK_ID,GENE_SYMBOL,...,UNIGENE_ID,ENSEMBL_ID,TIGR_ID,ACCESSION_STRING,CHROMOSOMAL_LOCATION,CYTOBAND,DESCRIPTION,GO_ID,SEQUENCE,SPOT_ID.1
0,1,192,328.0,GE_BrightCorner,GE_BrightCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,192,326.0,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,192,324.0,DarkCorner,DarkCorner,pos,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,192,322.0,A_21_P0014386,A_21_P0014386,FALSE,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,unmapped,NaN,NaN,NaN,AATACATGTTTTGGTAAACACTCGGTCAGAGCACCCTCTTTCTGTG...,NaN
4,5,192,320.0,A_33_P3396872,A_33_P3396872,FALSE,NM_001105533,NM_001105533,79974.0,CPED1,...,Hs.189652,NaN,NaN,ref|NM_001105533|gb|AK025639|gb|BC030538|tc|TH...,chr7:120901888-120901947,hs|7q31.31,Homo sapiens cadherin-like and PC-esterase dom...,GO:0005783(endoplasmic reticulum),GCTTATCTCACCTAATACAGGGACTATGCAACCAAGAAACTGGAAA...,NaN


In [26]:
condition_gene_name_col = 'GENE_SYMBOL'

In [27]:
condition_gene_mapping = get_gene_mapping(condition_gene_annotation, condition_gene_name_col)

In [28]:
condition_genetic_data = get_genetic_data(condition_matrix_file)

Load tabular gene expression data

In [29]:
condition_genetic_data = apply_gene_mapping(condition_genetic_data, condition_gene_mapping)

In [30]:
if NORMALIZE_GENE:
    condition_genetic_data = normalize_gene_symbols_in_index(condition_genetic_data)

25 input query terms found dup hits:	[('ACTA2-AS1', 2), ('ACTG1P20', 2), ('ACTG1P4', 2), ('ACTR3BP5', 3), ('ADAM1A', 2), ('ADCY10P1', 2),
55 input query terms found no hit:	['AAED1', 'AARS', 'ACN9', 'ACPP', 'ACRC', 'ADCK3', 'ADCK4', 'ADPRHL2', 'ADRBK1', 'ADRBK2', 'ADSS', '
13 input query terms found dup hits:	[('BCORP1', 2), ('BCRP2', 2), ('BMS1P17', 2), ('BMS1P20', 2), ('BRD7P3', 2), ('BTF3P11', 2), ('BTN2A
173 input query terms found no hit:	['BMS1P5', 'BMS1P6', 'BOLA3-AS1', 'BRE', 'BREA2', 'BTBD11', 'BZRAP1', 'BZRAP1-AS1', 'C10orf11', 'C10
21 input query terms found dup hits:	[('CLUHP3', 2), ('CMAHP', 2), ('CROCCP2', 2), ('CRYBB2P1', 2), ('CTAGE11P', 2), ('CTSLP2', 2), ('CTS
37 input query terms found no hit:	['CIDECP', 'CIRH1A', 'CLECL1', 'COL4A3BP', 'COX10-AS1', 'CPSF3L', 'CRHR1-IT1', 'CRIPAK', 'CSRP2BP', 
16 input query terms found dup hits:	[('ECEL1P2', 2), ('EDNRB-AS1', 2), ('EEF1DP3', 2), ('EMC3-AS1', 2), ('EPB41L4A-AS1', 2), ('FAAHP1', 
133 input query terms found no hit:	['E

In [31]:
condition_merged_data = add_binary_feature(condition_genetic_data, condition_feature_data, CONDITION)
condition_merged_data = condition_merged_data.T
condition_merged_data

,A1BG,A1BG-AS1,A2M-AS1,A4GALT,AAAS,AACS,AADACL3,AAGAB,AAK1,AAMDC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,Obesity
GSM5494930,9.356389,6.588705,10.086590,6.087023,8.855058,6.453003,6.644548,9.746953,8.111067,8.766887,...,6.278003,5.738853,5.108455,9.036174,7.664560,6.691724,10.559644,11.370012,8.635409,0.0
GSM5494931,9.580217,6.861172,8.589913,5.958440,8.172307,6.070650,6.281522,9.658012,8.342455,8.749621,...,6.687131,6.101444,5.254791,9.805082,7.791538,6.815505,10.200364,11.444456,8.916923,1.0
GSM5494932,9.920784,7.055549,9.467662,6.690681,8.768802,6.703954,7.138168,9.096505,8.249213,8.939544,...,6.648450,6.406971,5.426840,8.937783,7.575047,6.709371,11.124261,11.062703,8.439625,0.0
GSM5494933,9.504974,6.792186,7.930585,5.814862,8.708854,6.803403,5.934699,9.902677,7.866595,8.747317,...,6.583358,6.144584,5.039489,9.203809,7.625925,6.840305,11.300428,10.987588,8.646433,1.0
GSM5494934,9.533504,7.192053,9.596064,5.822462,8.534389,6.751145,6.999438,8.589853,8.513349,8.718470,...,6.475069,6.905870,5.460082,9.331111,8.048430,6.860686,10.561634,10.786569,8.603074,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM5495003,9.838970,7.523552,9.046983,6.697867,8.848680,7.336287,6.809151,9.434334,7.995361,9.067806,...,6.873141,6.783006,5.983136,9.089674,7.871114,7.021404,11.141075,10.901566,8.563167,0.0
GSM5495004,9.807525,7.071518,8.999367,6.329302,8.456660,7.068365,6.736399,9.067750,8.219228,8.873996,...,7.067476,7.412928,6.428669,9.036143,7.618206,6.996061,10.418056,10.881511,8.823480,1.0
GSM5495005,9.834407,7.480286,8.712552,6.283686,8.722496,7.056443,6.972152,9.209239,8.263851,8.986600,...,6.786707,6.913737,6.136249,9.058625,7.993798,7.245367,11.027180,10.812370,8.419373,1.0
GSM5495006,9.777699,7.482379,9.702852,6.623779,8.856997,7.177422,7.021150,9.143579,8.284696,9.088322,...,6.735525,7.475593,6.136097,9.195990,8.090627,7.009050,10.948086,10.848902,7.923561,1.0


In [32]:
biased = judge_binary_variable_biased(condition_merged_data, CONDITION, 0.1, 10)
if biased:
    print(f"The distribution of the trait \'{CONDITION}\' in this dataset is severely biased.")
else:
    print(f"The distribution of the trait \'{CONDITION}\' in this dataset is fine.")

The least common label is '0.0' with 30 occurrences. This represents 38.46% of the dataset.
The distribution of the trait 'Obesity' in this dataset is fine.


## 2.3. Finding regressors for two-step regression

In [33]:
related_genes_file = os.path.join(data_root, 'Summary_Corresponding_Gene_Symbol.csv')
condition_related_genes = get_feature_related_genes(related_genes_file, CONDITION, NORMALIZE_GENE)
print(f"Genes related to the condition '{CONDITION}' according to domain knowledge, {condition_related_genes}")

Genes related to the condition 'Obesity' according to domain knowledge, ['LEP', 'PPARG', 'POMC', 'MC4R', 'ENPP1', 'ADCY3']


In [34]:
gene_regressors_for_condition, common_genes_across_data = get_gene_regressors(trait_genetic_data, condition_genetic_data, condition_related_genes)

The trait and condition datasets have 10299 genes in common, such as ['FBXW2', 'ASPHD2', 'TMEM170A', 'DNAJC14', 'EIF2AK1', 'TNFSF10', 'GORASP2', 'GIGYF2', 'PTK2', 'ZNF839'].
Found 2 candidate genes that can be used in two-step regression analysis, such as ['ADCY3', 'POMC'].


In [35]:
gene_regressors_for_condition is not None

True

Otherwise, choose a few genes from 'common_genes_across_data' which might be related to the condition according to your biomedical knowledge, and use them as 'gene_regressors_for_condition'

# 3. The First Stage Regression Analysis

### 3.1. Do regression over condition dataset

In [36]:
X_condition = condition_merged_data[gene_regressors_for_condition].values
y_condition = condition_merged_data[CONDITION].values

# X_condition, _ = normalize_data(X_condition)
cv_mean, cv_std = cross_validation_with_lasso(X_condition, y_condition)

print(f'The cross-validation accuracy is {(cv_mean * 100):.2f}% ± {(cv_std * 100):.2f}%')

The cross-validation accuracy is 64.00% ± 13.73%


In [37]:
# Select relevant columns and convert to numpy array
print("Common gene regressors for condition and trait", gene_regressors_for_condition)

normalized_X_condition, _ = normalize_data(X_condition)

model = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
model.fit(normalized_X_condition, y_condition)

Common gene regressors for condition and trait ['ADCY3', 'POMC']


LogisticRegression(penalty='l1', random_state=42, solver='liblinear')

### 3.2. Predict the condition in traits

In [38]:
# Select relevant columns and convert to numpy array
regressors_in_trait = trait_merged_data[gene_regressors_for_condition].values
normalized_regressors_in_trait, _ = normalize_data(regressors_in_trait)
predicted_condition = model.predict_proba(normalized_regressors_in_trait)[:, 1]

In [39]:
#Add the predicted condition to the gene data for trait
#trait_merged_data.insert(0, CONDITION, predicted_condition)
trait_merged_data[CONDITION] = predicted_condition
trait_merged_data = trait_merged_data.drop(columns=gene_regressors_for_condition)
trait_merged_data.head()

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAA1,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Epilepsy,Obesity
GSM6216198,0.498235,0.909286,0.803357,0.881167,0.454591,0.887167,0.9117,0.157538,0.706483,0.92025,...,0.460750,0.5251,0.332605,0.554000,0.519077,0.365947,0.522561,0.162056,1.0,0.672131
GSM6216199,0.479765,0.900571,0.831571,0.874278,0.437136,0.895333,0.9172,0.157769,0.702603,0.91275,...,0.236667,0.2783,0.323442,0.559333,0.517308,0.375368,0.512585,0.139722,1.0,0.690948
GSM6216200,0.490882,0.900000,0.814643,0.844056,0.437409,0.872500,0.9123,0.153846,0.703810,0.91425,...,0.229333,0.3173,0.331791,0.558600,0.508000,0.368316,0.517439,0.149056,1.0,0.671753
GSM6216201,0.527765,0.900857,0.771929,0.903000,0.440500,0.804333,0.8902,0.164000,0.725345,0.92125,...,0.448167,0.6069,0.332116,0.560267,0.528846,0.317737,0.517293,0.157000,1.0,0.697817
GSM6216202,0.485706,0.895857,0.776429,0.838000,0.438864,0.842500,0.8885,0.161385,0.705621,0.88225,...,0.398750,0.4872,0.325698,0.546733,0.519769,0.356000,0.514829,0.158444,1.0,0.695657


# 4. The Second Stage Regression Analysis

In [40]:
trait_feature_cols = trait_merged_data.columns.tolist()
trait_feature_cols.remove(TRAIT)

In [41]:
# Select relevant columns and convert to numpy array
X_trait = trait_merged_data.drop(columns=[TRAIT]).values
y_trait = trait_merged_data[TRAIT].values
cv_mean, cv_std = cross_validation_with_lmm(X_trait, y_trait)
print(f'The cross-validation accuracy is {(cv_mean * 100):.2f}% ± {(cv_std * 100):.2f}%')

The cross-validation accuracy is 57.78% ± 4.44%


In [42]:
trait_merged_data

,A1BG,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAA1,AAAS,AACS,AADAC,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,Epilepsy,Obesity
GSM6216198,0.498235,0.909286,0.803357,0.881167,0.454591,0.887167,0.9117,0.157538,0.706483,0.92025,...,0.460750,0.5251,0.332605,0.554000,0.519077,0.365947,0.522561,0.162056,1.0,0.672131
GSM6216199,0.479765,0.900571,0.831571,0.874278,0.437136,0.895333,0.9172,0.157769,0.702603,0.91275,...,0.236667,0.2783,0.323442,0.559333,0.517308,0.375368,0.512585,0.139722,1.0,0.690948
GSM6216200,0.490882,0.900000,0.814643,0.844056,0.437409,0.872500,0.9123,0.153846,0.703810,0.91425,...,0.229333,0.3173,0.331791,0.558600,0.508000,0.368316,0.517439,0.149056,1.0,0.671753
GSM6216201,0.527765,0.900857,0.771929,0.903000,0.440500,0.804333,0.8902,0.164000,0.725345,0.92125,...,0.448167,0.6069,0.332116,0.560267,0.528846,0.317737,0.517293,0.157000,1.0,0.697817
GSM6216202,0.485706,0.895857,0.776429,0.838000,0.438864,0.842500,0.8885,0.161385,0.705621,0.88225,...,0.398750,0.4872,0.325698,0.546733,0.519769,0.356000,0.514829,0.158444,1.0,0.695657
GSM6216203,0.464529,0.896857,0.788929,0.863778,0.444364,0.863500,0.9017,0.153462,0.685948,0.92375,...,0.239417,0.2949,0.322256,0.548467,0.514000,0.364158,0.513976,0.133444,1.0,0.750267
GSM6216204,0.543765,0.905286,0.796500,0.889667,0.452727,0.876833,0.9155,0.175077,0.650776,0.94700,...,0.435333,0.4943,0.327953,0.554667,0.507000,0.332474,0.512585,0.152444,0.0,0.497299
GSM6216205,0.530529,0.895000,0.821571,0.890611,0.449409,0.875167,0.9178,0.157308,0.665466,0.92750,...,0.243833,0.2748,0.323163,0.553467,0.500308,0.347474,0.516390,0.150389,0.0,0.534364
GSM6216206,0.541882,0.920143,0.805357,0.851278,0.459182,0.860833,0.9167,0.158615,0.658121,0.91650,...,0.461500,0.5138,0.336093,0.566867,0.526615,0.327684,0.525683,0.160889,0.0,0.496767
GSM6216207,0.558471,0.877429,0.852857,0.864000,0.434091,0.912667,0.9123,0.156769,0.709310,0.92175,...,0.257000,0.2312,0.346721,0.576800,0.506615,0.386579,0.515854,0.128111,0.0,0.637628


In [43]:
# Conduct regression on the whole dataset
model = VariableSelection()

normalized_X_trait, _ = normalize_data(X_trait)
model.fit(normalized_X_trait, y_trait)

## 5. Report results

In [44]:
report_result_from_lmm(model, trait_feature_cols, TRAIT, CONDITION, threshold=0.05, save_output=True, output_dir='./output')

Effect of the condition on the target variable:
Variable: Obesity
Coefficient: 0.4383
p-value: 0.0003098

Found 8 significant genes affecting the trait 'Epilepsy' conditional on the factor 'Obesity', with corrected p-value < 0.05:
Variable  Coefficient  corrected_p_value
    IRX2     0.332506           0.031522
   DHRS3    -0.466792           0.034699
    EDN1    -0.453385           0.034699
   FGFR3    -0.455604           0.034699
  MRGPRE    -0.432295           0.034699
  STEAP4    -0.500208           0.034699
  FKBP10    -0.467017           0.040775
    TBR1     0.401915           0.045670


/media/techt/One Touch/Gold-Standard-Pipeline-Guide/utils.py:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
